# API Keys

In [1]:
from config import METISAI_API_KEY

# Text Collector

In [2]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader, UnstructuredMarkdownLoader

### TEST

In [69]:
loader = TextLoader('../test.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=4)
docs = text_splitter.split_documents(documents)

### Markdown webpages

In [3]:
import os
from langchain_core.documents import Document

file_path = '../index/'

docs = []

for markdown_file_name in os.listdir(file_path):

    markdown_file_path = file_path+ markdown_file_name
    print(markdown_file_path)

    with open(markdown_file_path, "r") as f:
        content = f.read()
    
    if len(content) % 1000 > 0:
        content += " " * (1000 - len(content) % 1000)

    for i in range(0, len(content), 1000):
        chunk = content[i:i + 1000]
        # print(chunk)
        doc = Document(page_content=chunk, metadata={})
        docs.append(doc)

    # loader = TextLoader(markdown_file_path)
    # documents = loader.load()
    # text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=4)
    # docs.extend(text_splitter.split_documents(documents))


../index/b912e7edbd72570b1ed0031a1bbc6f38faeae2de.md
../index/313b011b4d7d81a2a0f9f94d85cfe0264bc38f94.md
../index/1e4d7014b5b180ba23d6cb617b1f327dd96d70d1.md
../index/5290c4a4fd6ad4d182bac6ed92bef36df516fa03.md
../index/696eb8c6ee73c08a4fef66a1357346d270395aca.md
../index/2b37a236ce823cab838c5a6e52170b4f985d3af1.md
../index/564a109792267a6be18c6266666622a2ad1d5bbe.md
../index/84f6b04d5bb4f2106ac7405130ce732482136376.md
../index/536e1fdb3d4e967e6f1b35327587ea856d0e9a03.md
../index/674bc6ddf49a1cdf9d411a4446ec9c9b6d4dcb26.md
../index/dccd7a981a537afd3778740187a8673cb4c57f3b.md
../index/18eb6e63d1c7f4b6a315f4df8693a74e7bcc450e.md
../index/f212a5d4b24cc95da73b962de6f3c19908d87dd6.md
../index/1e64b0f4d8d4837af203642bb2eb93a4baf8b173.md
../index/a4f666ab09f65e6d7db244ea1f1b1535e5d90126.md
../index/ca0e39d2656c823cba3d97cf6395298abbc12ca0.md
../index/581bd52aedcc5362a85c789287dc9302e5786777.md
../index/70a84798e4acc78009124a3f142637b3076534e8.md
../index/9318e7c553f88fbd06933e627228b8c753640

In [4]:
len(docs)

1147

In [7]:
# pip install unstructured
# pip install "langchain[unstructured]"

# from langchain.document_loaders import UnstructuredMarkdownLoader

# loader = UnstructuredMarkdownLoader("your_file.md")
# documents = loader.load()

# from langchain.text_splitter import RecursiveCharacterTextSplitter

# splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
# chunks = splitter.split_documents(documents)

# # Define the path to your Markdown file
# file_path = "..index/"

# markdown_file_path = "path/to/your/file.md"

# # Initialize the loader
# loader = UnstructuredMarkdownLoader(markdown_file_path)

# # Load the document(s)
# data = loader.load()


# # The 'data' variable will contain a list of LangChain Document objects.
# # If using 'single' mode (default), it will typically be a list with one Document.
# # If using 'elements' mode, it will contain multiple Documents representing elements like headers, paragraphs, etc.
# data

In [64]:
docs = docs[0:300]

In [65]:
len(docs)

300

In [33]:
docs[70].page_content

'در جدیدترین اخبار سینما و تلویزیون، اولین تریلر رسمی و کامل فصل دوم سریال Iron Fist(آیرون فیست) منتشر شده است. با اینکه فصل اول این سریال بسیاری را ناامید کرد، اما شبکه نت فلیکس و بخش تلویزیونی مارول فرصت دیگری را به این سریال دادند و . کمتر از یک ماه دیگر فصل دوم این سریال از شبکه نتفلیکس منتشر خواهد شد و پیش از این یک در ماه گذشته و یک در طول این ماه از فصل دوم سریال Iron Fist منتشر شده است. حال سرانجام اولین تریلر کامل فصل دوم این سریال منتشر شده که در آن شاهد تصاویر بسیاری از اتفاقات و شخصیت\u200cهای اصلی و شرور فصل دوم سریال هستیم. همانطور که در دو ویدیو قبلی نیز به آن اشاره شده بود، دنی رند باید با یک تهدید بسیار خطرناک از گذشته مواجه شود و داووس که در گذشته با دنی رند آموزش دیده بود، حال به یک تهدید بسیار خطرناک تبدیل شده و قدرت مشابه آیرون فیست را به\u200cدست آورده است. همچنین شخصیت تایفوید مری نیز در طول تریلر معرفی می\u200cشود و در این تریلر شاهد حضور شخصیت میستی نایت نیز هستیم. شما در ادامه می\u200cتوانید این تریلر را تماشا کنید:\nپیش از این در فصل دوم سریال Luke Cage (لوک 

# Embedings

# Vector Store

In [7]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from pydantic import SecretStr

# Set UP the link
vector_store = Chroma(
    collection_name="test",
    embedding_function=OpenAIEmbeddings(
        model="text-embedding-3-small",
        api_key=SecretStr(METISAI_API_KEY),
        base_url="https://api.metisai.ir/openai/v1",
    ),
)

for i in range(int(len(docs) / 250 + 1)):
    vector_store.add_documents(docs[i*250: (i+1)*250], ids=[str(j + i * 250) for j in range(len(docs))])

/tmp/ipykernel_11754/1392131427.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [8]:
vector_store._collection.count()

1147

# LLM

In [21]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4.1-nano",
    api_key=SecretStr(METISAI_API_KEY),
    base_url="https://api.metisai.ir/openai/v1",
    temperature=0.1,
    verbose=True,
)

In [29]:
a = llm.invoke("موسیقی متن سریال Euphoria را چه کسی ساخته است؟")

In [30]:
print(a.content)

موسیقی متن سریال Euphoria توسط آهنگساز آمریکایی لودویگ غروبر ساخته شده است.


# Prompt

In [10]:
from langchain import PromptTemplate

# template = """
# You are a fortune teller. These Human will ask you a questions about their life.
# Use following piece of context to answer the question.
# If you don't know the answer, just say you don't know.
# Keep the answer within 2 sentences and concise.

# Context: {context}
# Question: {question}
# Answer:

# """

template = """
لطفا بر اساس مطلب داده شده به سوال پاسخ بده

مطلب: {context}
سوال: {question}
جواب:
"""

prompt = PromptTemplate(
  template=template,
  input_variables=["context", "question"]
)

In [11]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
  {"context": vector_store.as_retriever(),  "question": RunnablePassthrough()}
  | prompt
  | llm
  | StrOutputParser()
)

In [12]:
result = rag_chain.invoke(" فیلم jurrasic world در چه کشوری اکران خواهد شد؟ و در چه تاریخی؟")
print(result)

فیلم Jurassic World در آمریکا اکران خواهد شد و تاریخ اکران آن هفته آینده است.


In [13]:
result = rag_chain.invoke(" شخصیت Iron Fist در چه سریالی بازی کرده است؟")
print(result)

شخصیت Iron Fist در سریال "Iron Fist" بازی کرده است.


In [85]:
result = rag_chain.invoke(" چند فصل از سریال Iron Fist تولید شده است؟")
print(result)

دو فصل از سریال Iron Fist تولید شده است.


In [18]:
result = rag_chain.invoke("بازی love letter برای چه رده سنی ای است؟")
print(result)

بازی Love Letter برای افراد بالای ۱۰ سال طراحی شده است.
